In [13]:
def example_strategy_step_function(elevator, state):
    """
    The function implements the strategy of an elevator starting at the ground floor and moving to the last floor, stoping at each floor.
    
    Parameters:
        elevator (Elevator instance): 
        state (dict): The dictionary that is keeping the track of the state, in this case the direction in which elevator is moving
    """
    
    MOVING_UP = "moving_up"
    STOPPED_AND_OPENED = "stopped_and_opened"
    if not state:
        state[MOVING_UP] = True
        state[STOPPED_AND_OPENED] = False

    state[STOPPED_AND_OPENED] = not state[STOPPED_AND_OPENED]

    if state[STOPPED_AND_OPENED]:
        return Simulation.STOP_AND_OPEN
    else:
        current_floor = elevator.current_floor
        top_floor = elevator.floor_range[-1]
        bottom_floor = elevator.floor_range[0]

        if state[MOVING_UP] == True and current_floor == top_floor:
            state[MOVING_UP] = False
        elif state[MOVING_UP] == False and current_floor == bottom_floor:
            state[MOVING_UP] = True

        if state[MOVING_UP] == True:
            return Simulation.MOVE_UP
        else:
            return Simulation.MOVE_DOWN

modeling = Simulation(no_of_floors=100,no_of_passengers=1000, capacity=15, step_function=example_strategy_step_function)

('Average number of extra steps taken is ', 4545.048)
('The 95th percentile of extra steps taken is ', 8820)
('Total number of steps is ', 9990)


In [14]:
def first_come_step_function(elevator,state):
    """
    Defines the "First Come, First Serve" elevator strategy.

    Parameters:
        elevator (Elevator object):
        state (dict): The dictionary that is keeping the track of the state, in this case the direction in which elevator is moving.
    """
    if not elevator.passengers_in_elevator:
        if elevator.current_floor < elevator.pending_requests[0].starting_floor:
            return Simulation.MOVE_UP
        elif elevator.current_floor > elevator.pending_requests[0].starting_floor:
            return Simulation.MOVE_DOWN
        else:
            return Simulation.STOP_AND_OPEN
    else:
        if elevator.current_floor < elevator.passengers_in_elevator[0].destination_floor:
            return Simulation.MOVE_UP
        elif elevator.current_floor > elevator.passengers_in_elevator[0].destination_floor:
            return Simulation.MOVE_DOWN
        else:
            return Simulation.STOP_AND_OPEN

modeling = Simulation(no_of_floors=100,no_of_passengers=1000, capacity=15, step_function=first_come_step_function)


('Average number of extra steps taken is ', 15339.618)
('The 95th percentile of extra steps taken is ', 29367)
('Total number of steps is ', 32307)


In [15]:
from collections import Counter


def democratic_strategy_step_function(elevator, state):
    """Defines the Democratic Strategy, picking up passengers who are on floors the elevator is passing, and dropping off passengers at the floor most have as their destination.

    Parameters:
        elevator (Elevator object):
        state (dict): The dictionary that is keeping the track of the state, in this case the direction in which elevator is moving.
    """

    for passenger in elevator.passengers_in_elevator:  # Checks if we are on the destination of someone.
        if passenger.destination_floor == elevator.current_floor:
            return Simulation.STOP_AND_OPEN

    if (len(elevator.passengers_in_elevator) < elevator.capacity  # Checks if we can pick up people on
                                                                  # current floor.
       and elevator.current_floor in elevator.requests_by_floor
       and elevator.requests_by_floor[elevator.current_floor]):
        return Simulation.STOP_AND_OPEN

    floors = None
    if elevator.passengers_in_elevator:  # This block of code decides how to moves the elevator.
        floors = [ passenger.destination_floor for passenger in elevator.passengers_in_elevator ]
    else:
        floors = [ passenger.starting_floor for passenger in elevator.pending_requests ]
    mode_floor = Counter(floors).most_common(1)[0][0]
    return [ Simulation.MOVE_UP, Simulation.MOVE_DOWN, ][mode_floor < elevator.current_floor]

modeling = Simulation(no_of_floors=100,no_of_passengers=1000, capacity=15, step_function=democratic_strategy_step_function)

('Average number of extra steps taken is ', 3883.51)
('The 95th percentile of extra steps taken is ', 7640)
('Total number of steps is ', 8854)
